In [ ]:
import random
import pandas as pd
# import pmdarima
from pmdarima import auto_arima
# import prophet
# import statsmodels
from statsmodels.tsa.holtwinters import ExponentialSmoothing
# import sklearn

from prophet import Prophet
from statsmodels.tsa.holtwinters import ExponentialSmoothing
# from pmdarima import auto_arima
from sklearn.metrics import mean_squared_error

def time_series_models_comparison():
    random.seed(123)
    data_set = pd.read_csv("./data/temperatures.csv")
    # do not remove the lines above and write your solution here

    # Step 1: Prepare the dataset
    # Set the 'ds' column to a datetime type and create a date range
    data_set['ds'] = pd.to_datetime(data_set['ds'])
    date_range = pd.date_range(start="1980-01-01", end="2012-12-01", freq='MS')

    # Reindex the DataFrame to fill missing dates
    data_set = data_set.set_index('ds').reindex(date_range).reset_index()

    # Fill missing 'y' values forward
    data_set['y'].fillna(method='ffill', inplace=True)

    # Step 2: Split the dataset into training and test sets
    trainset = data_set[:-24]
    testset = data_set[-24:]

    # Step 3: Fit the models
    arima_model = auto_arima(trainset['y'], seasonal=True, m=12, stepwise=True)
    prophet_model = Prophet(seasonality_mode='additive')
    prophet_model.fit(trainset)
    ex_smooth_model = ExponentialSmoothing(trainset['y'], seasonal='add', seasonal_periods=12)
    ex_smooth_model = ex_smooth_model.fit(optimized=True)

    # Step 4: Generate forecasts and calculate MSE
    arima_forecasts = arima_model.predict(n_periods=24)
    prophet_forecasts = prophet_model.make_future_dataframe(periods=24, freq='MS')
    prophet_forecasts = prophet_model.predict(prophet_forecasts)
    ex_smooth_forecasts = ex_smooth_model.forecast(steps=24)

    arima_mse = mean_squared_error(testset['y'], arima_forecasts)
    prophet_mse = mean_squared_error(testset['y'], prophet_forecasts[-24:])
    ex_smooth_mse = mean_squared_error(testset['y'], ex_smooth_forecasts)

    # Build the results dictionary
    results = {
        'trainset': trainset,
        'testset': testset,
        'forecasts': {
            'arima': arima_forecasts,
            'prophet': prophet_forecasts,
            'ex_smooth': ex_smooth_forecasts
        },
        'errors': {
            'model': ['arima', 'prophet', 'ex_smooth'],
            'mse': [arima_mse, prophet_mse, ex_smooth_mse]
        }
    }

    return results